An overview of the German electricity prices
=======

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
!pip install -q statsmodels
x_ai_path = (os.path.abspath(os.path.join("", os.pardir)))
germany_csv_path = os.path.join(x_ai_path, "Data", "Data", "Misc", "Germany.csv")
weather_csv_path = os.path.join(x_ai_path, "Data", "Data", "Weather", "weather.csv")

df_weather = pd.read_csv(weather_csv_path)
df_weather['date'] = pd.to_datetime(df_weather['date'])
df_weather = df_weather.sort_values(by='date')
df_weather.index = pd.to_datetime(df_weather['date'])

df_germany = pd.read_csv(germany_csv_path)
df_germany['Datetime (UTC)'] = pd.to_datetime(df_germany['Datetime (UTC)']) # Wir konvertieren das datum in ein Datenformat
df_germany = df_germany.sort_values(by='Datetime (UTC)')
df_germany.index = pd.to_datetime(df_germany['Datetime (UTC)'])
df_germany.set_index('Datetime (UTC)')

## Initial observations
The SMARD Data for Germany contains hourly prices ranging from the first of January 2015 to the first of April, making up a total of 81'073 data points. Inspecting the price given in €/MWhe, various interesting observations can be made:
- Negative electricity prices exist. This occured a total of 1606 times, with the price being exactly 0€ 65 times. 
- The lowest recorded price in the data set was -500€, occuring at 2022-08-23 20:00:00. The second lowest price with -399€ was achieved the same day. These are stark outliers, with only 13 entries below the -100€ mark. The sum being exactly 500.00€ in particular stands out as peculiar.
- The highest recorded price in the data set was 1047.11€, occurring at  2023-07-02 14:00:00. In total, only 5 entries surpass the 900€ mark, all of them past 2023. 

In [ ]:
print(df_germany.sort_values(by=['Price (EUR/MWhe)']))

# Negative values
## Explanation
Negative wholesale electricity prices are a phenomenon that arises when high and inflexible electricity generation, frequently from renewable sources such as wind and solar, coincides with low demand, particularly during periods of low consumption such as holidays. During these times, producers are remunerated for offloading excess electricity, which results in negative prices. This typically occurs when renewable generation exceeds demand, as evidenced by the case of Easter Monday 2019 in Germany, where the country's entire electricity needs were met by renewable energy production.
Further information can be found on the smard website itself.
https://www.smard.de/page/en/wiki-article/5884/105426

## Example
The lowest recorded price point in the dataset occured on 2023-07-02 12:00, with the oddly specific price of -500€/MWhe. At this point, renewable energy sources introduced 49.7GWh of energy into the system, with a total consumption of only 46.1 GWh. Comperatively, conventional plants produced only 7.1 GWh of electricity at that hour, with 4.1 GWh of electricity being stored by pumped storage power plants
https://www.ffe.de/en/publications/german-electricity-prices-on-epex-spot-2023/



In [ ]:
lowest_day_frame = df_germany.loc['2023-07-02 00:00:00':'2023-07-02 23:00:00']
lowest_day_weather = df_weather.loc['2023-07-19 22:00:00':'2023-07-20 22:00:00'] # weather.csv does not use UTC yet
hourly_price_array = lowest_day_frame.groupby(lowest_day_frame["Datetime (UTC)"].dt.hour)['Price (EUR/MWhe)'].mean()
hourly_sunshine_array = lowest_day_weather.groupby(lowest_day_weather["date"].dt.hour)['sunshine_min_Brocken'].mean()
average_sunshine_array = df_weather.groupby(df_weather["date"].dt.hour)['sunshine_min_Brocken'].mean()
#This plotting is not accurate yet, as the time is still not in UTC for the weather data

#replacing Nightime NaN values with 0 for plotting
hourly_sunshine_array = [0.0 if np.isnan(x) else x for x in hourly_sunshine_array]

plt.figure(figsize=(8,4))
hours = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

plt.plot(hours, hourly_price_array, label='Price in €/MWhe')
plt.plot(hours, hourly_sunshine_array, label='sunshine Minutes on Brocken weather station that day')
plt.plot(hours, average_sunshine_array, label='average sunshine on Brocken')
plt.grid(color='#95a5a6', linestyle='--', linewidth=1, axis='y', alpha=0.5)
plt.legend()
plt.title('Hourly electricity price on the 2nd of July 2023')
plt.show()

import os
x_ai_path = (os.path.abspath(os.path.join("", os.pardir)))
germany_csv_path = os.path.join(x_ai_path, "Data", "Data", "Misc", "Germany.csv")


## Initial Plotting
An initial plotting of the entire data immediatly draws attention to the uniformity of the graph prior to the second half of 2021 compared to the much larger variance in the latter part of the diagram. There are also notable individual spikes, again particularly from the latter half of 2021 onwards.
This might be of particular relevance when trying to predict future prices.  

In [ ]:
df_germany['Year-Month'] = df_germany['Datetime (UTC)'].dt.to_period('M').dt.to_timestamp()
monthly_mean_frame = df_germany.groupby('Year-Month')['Price (EUR/MWhe)'].mean()
monthly_mean_frame = monthly_mean_frame.reset_index()

plt.figure(figsize=(12,6))
plt.plot(df_germany['Datetime (UTC)'], df_germany['Price (EUR/MWhe)'], label='Price')
plt.plot(monthly_mean_frame['Year-Month'], monthly_mean_frame['Price (EUR/MWhe)'], label='Monthly avg. price')
plt.plot()
plt.legend()
plt.title('Preis EUR/MWhe')
plt.show()


# Smaller Trends with seasonal decompose
However, at this scale, the data might not be very indicative. To gain further insight, a simple decomposition is done on the span of the last four weeks. 
Here, the seasonal window clearly shows each day, with 28 distinct repeating patterns, showcasing the individual days and indicating a strong dailiy cycle. The trend window on the other hand, indicates a lowered prices on weekends. 

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
series = df_germany['Price (EUR/MWhe)']
#24 *7 * 4 = 672 
result = seasonal_decompose(series[-672:], model='additive')
result.plot()
plt.show()


## Plotting these trends

### Throughout the day
A plot of the electricity price throughout the course of the day reveals two peaks, occurring at approximately 6 a.m. and 5 p.m., respectively. Additionally, the price reaches its lowest point during the night, at 2 a.m. This indicates a pronounced decline in energy consumption, as the production of energy from solar sources is also reduced during nocturnal hours. Conversely, solar production is most active between the hours of 11 a.m. and 4 p.m.

In [ ]:
#Primitive first plotting
weekday_mean_frame = df_germany.groupby(df_germany["Datetime (UTC)"].dt.hour)['Price (EUR/MWhe)'].mean()

plt.figure(figsize=(8,4))
hours = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

plt.plot(hours, weekday_mean_frame, label='Price')
plt.grid(color='#95a5a6', linestyle='--', linewidth=1, axis='y', alpha=0.5)
plt.legend()
plt.title('Average electricity price in €/MWhe per hour of the day since 2015')
plt.show()




# Boxplot with seaborn
df_germany['Hour'] = df_germany['Datetime (UTC)'].dt.hour
plt.figure(figsize=(8, 4))
sns.boxplot(x='Hour', y='Price (EUR/MWhe)', data=df_germany, color='lightblue')

plt.title('Distribution of Electricity Price per Hour of the Day since 2015', fontsize=14)
plt.xlabel('Hour of the Day')
plt.ylabel('Price (EUR/MWhe)')
plt.grid(True, axis='y', linestyle='--', linewidth=1, alpha=0.5)
plt.xticks(range(0, 24))

plt.show()



#Median and interquartile range with pyplot
df_filtered = df_germany[(df_germany['Price (EUR/MWhe)'] >= -500) & (df_germany['Price (EUR/MWhe)'] <= 500)]
summary = df_filtered.groupby('Hour')['Price (EUR/MWhe)'].describe()
# Plot median and interquartile range
plt.figure(figsize=(10, 6))
plt.plot(summary.index, summary['50%'], label='Median', marker='o', color='blue')  # Median
plt.fill_between(summary.index, summary['25%'], summary['75%'], color='blue', alpha=0.3, label='IQR (25%-75%)')  # IQR
plt.plot(summary.index, summary['mean'], label='Mean', color='black')

plt.title('Hourly Electricity Prices (Filtered, ±500 EUR/MWh)', fontsize=14)
plt.xlabel('Hour of the Day', fontsize=12)
plt.ylabel('Price (EUR/MWhe)', fontsize=12)
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.xticks(range(0, 24))
plt.legend()

plt.tight_layout()
plt.show()



# Heatmap
hourly_mean = df_germany.groupby(df_germany["Datetime (UTC)"].dt.hour)['Price (EUR/MWhe)'].mean()

hourly_mean_df = pd.DataFrame(hourly_mean).reset_index()
hourly_mean_df = hourly_mean_df.rename(columns={'Datetime (UTC)': 'Hour', 'Price (EUR/MWhe)': 'Average Price'})

heatmap_data = hourly_mean_df[['Average Price']].T

plt.figure(figsize=(12, 2))
sns.heatmap(heatmap_data, cmap="coolwarm", annot=True, fmt=".1f", cbar_kws={'label': 'Price (EUR/MWhe)'}, xticklabels=hourly_mean_df['Hour'], yticklabels=[''])

plt.title('Heatmap of Average Electricity Prices per Hour of the Day', fontsize=14)
plt.xlabel('Hour of the Day')
plt.tight_layout()

plt.show()


### Throughout the week
Secondly, let us examine the daily trend over the course of a week. By averaging the values of the entire data set per weekday, it becomes evident that there is a clear downward trend in prices over the weekend, with Sunday in particular being a low point. This can be speculated to be related to lowered consumption. 

In [ ]:
weekday_mean_frame = df_germany.groupby(df_germany["Datetime (UTC)"].dt.weekday)['Price (EUR/MWhe)'].mean()

plt.figure(figsize=(8,4))
plt.bar(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], weekday_mean_frame, label='Price')
plt.grid(color='#95a5a6', linestyle='--', linewidth=1, axis='y', alpha=0.5)
plt.legend()
plt.title('Average electricity price in €/MWhe per weekday since 2015')
plt.show()

### Throughout the months of the year
Averaging the months of the year, a further difference is visible. This can be speculated to both pertain to temperatures and therefore heating and weather influences. 

In [ ]:
monthly_mean_list = df_germany.groupby(df_germany["Datetime (UTC)"].dt.month)['Price (EUR/MWhe)'].mean()

plt.figure(figsize=(12,4))
plt.bar(["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"], monthly_mean_list, label='Price')
plt.grid(color='#95a5a6', linestyle='--', linewidth=1, axis='y', alpha=0.5)
plt.legend()
plt.title('Average electricity price in €/MWhe per month since 2015')
plt.show()

### Years
Averaging the individual years, a striking spike during the year 2022 stands out. A slight increase due to inflation is to be expected, however the spike clearly superceeds the inflation of 7,9% that year. Inflation data is taken from https://www.finanz-tools.de/inflation/inflationsraten-deutschland

In [ ]:
df_germany.set_index('Datetime (UTC)')
yearly_mean_frame = df_germany.groupby(df_germany["Datetime (UTC)"].dt.year)['Price (EUR/MWhe)'].mean()

plt.figure(figsize=(8,6))
plt.bar([2015,2016,2017,2018,2019,2020,2021,2022,2023,2024], yearly_mean_frame, label='Price')
plt.legend()
plt.title('Average electricity price in €/MWhe per year since 2015')
plt.show()

previous_year_price = yearly_mean_frame[2015]
price_changes = []
for yearly_price in yearly_mean_frame:
    if yearly_price != yearly_mean_frame[2015]:
        price_changes.append (yearly_price / previous_year_price * 100 -100)
        previous_year_price = yearly_price

inflation_from_2016 = [0.5, 1.5, 1.8, 1.4, 0.5, 3.1, 6.9, 5.9, 2.2]
plt.figure(figsize=(8,6))
plt.bar([2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], price_changes, width=0.5,label='price change in %')
plt.bar([2016+0.5, 2017+0.5, 2018+0.5, 2019+0.5, 2020+0.5, 2021+0.5, 2022+0.5, 2023+0.5, 2024+0.5], inflation_from_2016, width=0.5,label='inflation in %')
plt.legend()
plt.title('Average change in price in % compared to the previous year since 2016')
plt.show()

# Political Influences and Events
The European electricity market is a regulated one. Unfortunately for us trying to predict the future, real world politics and occurences are subject to change. 

In [ ]:
date_strings = ["2018-10-01", "2020-03-11", "2022-02-24", "2023-03-01", "2023-12-31"]
events = ["Split of German/Austrian \n shared market", "COVID-19 declared\n a Pandemic", "Full scale Russian \n invasion of Ukraine", "Start of Strompreisbremse", "End of Strompreisbremse"]
df_events = pd.DataFrame(data={"Date": date_strings, "Event": events})
df_events["Date"] = pd.to_datetime(df_events["Date"])
#Adding a random height for plotting on the graph later
df_events["Level"] = [np.random.randint(-6,-2) if (i%2)==0 else np.random.randint(2,6) for i in range(len(df_events))]
df_events["Level"] = df_events["Level"]*200

fig, ax = plt.subplots(figsize=(15,7))
ax.plot(df_events.Date, [0,]* len(df_events), "-o", color="black", markerfacecolor="white")
ax.set_xticks(pd.date_range("2015-1-1", "2025-1-1", freq="ys"), range(2015, 2026));
ax.set_ylim(-600,1050);
for idx in range(len(df_events)):
    date, event, level = df_events["Date"][idx], df_events["Event"][idx], df_events["Level"][idx]
    date_string = date.strftime("%b-%Y")
    ax.annotate(date_string + "\n" + event, xy=(date, 0.1 if level>0 else -0.1),xytext=(date, level), arrowprops=dict(arrowstyle="-",color="red", linewidth=0.8), ha="center")

ax.spines[["left", "top", "right", "bottom"]].set_visible(False);
ax.spines[["bottom"]].set_position(("axes", 0.5));
ax.plot(monthly_mean_frame['Year-Month'], monthly_mean_frame['Price (EUR/MWhe)'], label='Monthly avg. price')
ax.yaxis.set_visible(False);
ax.set_title("Events", pad=10, loc="left", fontsize=25, fontweight="bold");


# Limitations of the Data

## Market change
The precise definition of the German energy price has undergone alterations throughout the entirety of the data set. In particular, it is necessary to determine which entities are included and which are excluded.
Up until October 2018, Germany constituted a unified market with both Luxembourg and Austria. 
However, in a move that was met with disapproval by the Austrian government, the integrated market was dismantled as a result of political decisions and now only includes Germany and Luxembourg.
See https://www.politico.eu/article/germany-austria-energy-europe-power-couple-break-up/

## Strompreisrbremse 
The "Strompreisbremse" was a government measure introduced in Germany with the objective of limiting electricity costs. Its implementation was driven by the need to mitigate the economic and social consequences of the war in Ukraine. For households and small businesses, the electricity price is capped at 40 cents per kilowatt hour for 80% of the previous year's consumption, while medium-sized and large businesses pay a net price of 13 cents for 70% of their consumption. The measure is financed by the Economic Stabilization Fund. Following a ruling by the Federal Constitutional Court, it was decided that this and similar subsidies would be phased out at the end of 2023, as financing through the reallocation of corona funds was deemed unconstitutional.
https://de.wikipedia.org/wiki/Strompreisbremse

## Erroneous calculations
On June 26, 2024, the day-ahead price was not correctly calculated, resulting in a lower price than it should have been. This affected customers using dynamic pricing tariffs to a significant extent. 
See https://de.wikipedia.org/wiki/EPEX_Spot_Dayahead_Auktion#Vorkommnis
